<div style="border-style: solid;border-width: 1px;">
<div style="height: 200px; 
            width: 750px; 
            color: #ffffff;
            margin: auto;
	        background-color: #ff6600; 
	        background-image: url(http://icarus.lngs.infn.it/img/n3.jpg); 
            background-repeat: no-repeat;
            background-position: center;">
</div>

<div style="text-align:center">
    <h1><font color="blue"><font size="6">Study of 2D Noise Filtering Techniques in ICARUS</font></font></h1><br>
    <font color="gray"><font size="3">A jupyter notebook aimed at studying 2D Noise Filtering with the ICARUS LAr TPC</font></font><br>
</div>
</div>
<br>
<h3>What this notebook intends to do:</h3>
<ul>
    <li>Operates on art root format data files that contain raw::RawDigit objects</li>
        <ul>
            <li>The RawDigits contain waveforms representing the TPC readout </li>
        </ul>
    <li>Once the RawDigits have been recovered the idea is to filtering techniques</li>
        <ul>
            <li>Look at the standard coherent noise subtraction with eye towards how to best "protect" signal</li>
            <li>Explore some promising 2D techniques for identifying possible signal</li>
        </ul>
</ul>

<h3>Some useful references</h3>
<ul>
    <li>A detailed desription of the electronics to be used for ICARUS at SBN can be found in the <a href="https://iopscience.iop.org/article/10.1088/1748-0221/13/12/P12007/pdf">electronics paper</a> which includes performance tests from CERN. From this paper we gather a useful conversion of rms ADC counts to ENC is ~3 e-/ADC for "intrinsic" noise</li>
    <li>The paper detailing the Design, construction and tests of the ICARUS T600 detector can be found <a href="https://pdfs.semanticscholar.org/f69f/74169f2dfa8a15981d1d70aecd5c7aa82831.pdf">here</a>. Section 7 details the TPC wires with Table 6 giving an overview (including wire lengths), section 7.5 details the cabling. Section 11 describes the readout electronics, with section 11.2 describing the input capacitances for each of the planes (~400 pF for horizontal wires, ~200 pF for the U/V wires). </li>
</ul>

<h3>What is needed to run this notebook:</h3>
<ul>
    <li><p>Necessary python libraries/includes</p>
        <ul>
            <li> matplotlib   - the basic plotting package though note that here we mostly use plot-ly</li>
            <li> numpy        - python array handling and loads of useful functions</li>
            <li> scipy        - where we get the analytic functions we will make heavy use of </li>
            <li> ipywidgets   - useful for making interactive output</li>
            <li> uproot       - see below</li>
            <li> plotly       - the plot-ly graphing package which allows interactive plots</li>
        </ul></li>
    <li>And, of course, the <b>sigproc_tools</b> package from which this notebook was obtained</li>
    <li>uproot<br>
        In this notebook we use "uproot" to handle the root input. For more information on uproot follow this <a href="https://uproot.readthedocs.io/en/latest/">link</a></li>
</ul>



<div style="text-align:center;border-style: solid;border-width: 1px;">
    <h2><font color="blue"><font size="5">Start Up Block</font></font></h2><br>
</div>

<ul>
    <li>Import of standard libraries used below</li>
    <li>Set the system path to include looking in the package of noise handling functions/objects</li>
    <li>We define two variables which define the location of the input file</li>
        <ul>
            <li> PATHNAME       - Fully qualified path to the folder containing the input file</li>
            <li> RECOFILENAME   - the fully qualified file name for the input data file</li>
        </ul>
    <li>With these definitions we can open and read the input data file</li>
</ul>

In [ ]:
#
# Standard set up block to start with
#
# Probably most people will simply want "matplotlib inline" but on a mac it is useful to do the other stuff
%matplotlib inline
##matplotlib qt
##matplotlib notebook
##config InlineBackend.figure_format ='retina'
#%pylab

# This allows reading of root files without having to actually use root. Nice!
import uproot

# Useful stuff
import numpy as np
import math
import plotly.graph_objects as go
import plotly.subplots as subplots

from plotly.colors import DEFAULT_PLOTLY_COLORS

###############################################################################
# TODO:
# Set the path to our library of functions/objects/etc
import sys

sigProcPath = "/home/usher/LArTPC/icarus-sigproc-tools"
sys.path.insert(0,sigProcPath)

noiseLibPath = "~/LArTPC/signal_processing/build/lib"
sys.path.insert(1,sigProcPath)

###############################################################################
# TODO:
# Set the path and name of the data file to be read
PATHNAME       = "/home/usher/LArTPC/ICARUS/workarea/data"
RECOFILENAME   = PATHNAME + "/data_run621_1_EW05_PED_20190926T201608_DECODE.root"
#RECOFILENAME   = PATHNAME + "/data_run618_2_EW01M_PED_20190926T174904_DECODE.root"

<div style="text-align:center;border-style: solid;border-width: 1px;">
    <h2><font color="blue"><font size="5">Open, Read and Process the Input File</font></font></h2><br>
</div>

<ul>
    <li>We define three variables which should be common to all files
        <ul>
            <li> RECOFOLDERNAME - the name of the folder for the particular data set we want to analyze</li>
            <li> DAQNAME.       - currently unused</li>
        </ul></li>
    <li>With these definitions we can open and read the input data file</li>
    <li>Contents of the input file are read into a numpy array</li>
    <li>Once read in, then go ahead and process the input file with the standard noise filter</li>
</ul>

In [ ]:
from sigproc_tools.sigproc_objects.filterevents import FilterEvents
import LiteFMWK_algorithms

# Below should be standard for the test data files currently available
RECOFOLDERNAME = "Events" 
DAQNAME        = "raw::RawDigits_daq__TPCANALYSIS."

# Grab the pandas dataframe from the input file for the tree we want to look at
print("Opening file: ",RECOFILENAME)
data_file = uproot.open(RECOFILENAME)

print(data_file.keys(),"\n------\n")

print("Opening the folder contianing the RawDigits information: ",RECOFOLDERNAME)
events_folder = data_file[RECOFOLDERNAME]

print(events_folder)

# Go ahead and filter the events
filter = FilterEvents(events_folder,DAQNAME)

# What are the basic operating parameters?
nChannelsPerGroup = 64   # <==== We choose this
numChannels       = filter.rawdigits.numChannels(0)
numGroups         = numChannels // nChannelsPerGroup

numEvents = filter.filterEvents(nChannelsPerGroup)

print("Noise processing complete with",numEvents)

<div style="text-align:center;border-style: solid;border-width: 1px;">
    <h2><font color="blue"><font size="5">Create a Fake Particle and Overlay on Test Data</font></font></h2><br>
</div>

<ul>
    <li>We can take advantage of the implementation of the TPC field and electronics response functions to create a fairly realistic particle trajectory to overlay onto one of our data events.</li>
        <ul>
            <li>We can choose the number of electrons deposited, where this would be assumed to be after recombination, lifetime and any other effects which would reduce them - we're not doing a full drift simulation here!</li>
            <li>We specify the wire and tick ranges for the particle (within the existing data block)</li>
        </ul>
    <li>Overlay (add) this to an existing waveform with noise</li>
    <li>Optionally, if you set "makePlots" to True then you can view a 2D event display</li>
</ul>

In [ ]:
from plotting.graphs import *
from sigproc_tools.sigproc_objects.fullresponse import FullResponse
from sigproc_tools.sigproc_objects.fieldresponse import FieldResponse
from sigproc_tools.sigproc_objects.electronicsresponse import ElectronicsResponse
from sigproc_tools.sigproc_functions.fakeParticle import genWhiteNoiseWaveform,genSpikeWaveform,createParticleTrajectory

# Set up to read and initialize the response functions for each plane
inputFilePath = "/home/usher/LArTPC/ICARUS/workarea/data/"

TPCresponses = [None,None,None]

TPCresponses[2] = FullResponse(inputFilePath,"t600_response_vw02_v0.0.root")
TPCresponses[1] = FullResponse(inputFilePath,"t600_response_vw01_v0.0.root",TPCresponses[2].FieldResponse.normFactor)
TPCresponses[0] = FullResponse(inputFilePath,"t600_response_vw00_v0.0.root",TPCresponses[2].FieldResponse.normFactor)

# Experiment with creating an event pic
# Start with "just" a charge deposit on a zero waveform
numElectrons = 6300 # number electrons per mm after recombination
numChannels  = 576
numTicks     = 4096

angleToWire  = 90 # degrees
slope        = math.tan(math.radians(90-angleToWire)) / 0.213
startTick    = 2000

startWire    = 50
stopWire     = 550

print("Angle to wire:",angleToWire,"(deg), tan(theta):",math.tan(math.radians(90-angleToWire)),", slope:",slope)

wireRange = (startWire,stopWire)
tickRange = (startTick,int(round(slope*(wireRange[1]-wireRange[0])+startTick)))

print("Using wireRange:",wireRange,", tickRange:",tickRange)

spikeResponse = createParticleTrajectory(TPCresponses[2],numElectrons,wireRange,tickRange,(numChannels,numTicks))

# Example of how to produce a "white noise" waveform for background
#whiteRMSRaw = 5.0
#whiteResponse,whiteNoise = genWhiteNoiseWaveform(TPCresponses[1],whiteRMSRaw,(numChannels,numTicks))

# Here we overlay onto our data
fullEvent = spikeResponse + filter.waveLessPedAll[0,:,:]

print("Overlay event created, wires from:",wireRange,", ticks from:",tickRange)


In [ ]:
makePlots = True

if makePlots:
    overlayPic = plotEventView3D(fullEvent)
    print("Calling the show method")
    overlayPic.show()
    

<div style="text-align:center;border-style: solid;border-width: 1px;">
    <h2><font color="blue"><font size="5">Explore the various 2D filter tools</font></font></h2><br>
</div>

In [ ]:
#Basically, run everything we can think of...
import scipy.ndimage as ndimage
from sigproc_tools.sigproc_functions.noiseProcessing import *

#filteredFullEvent,filteredMedian,filteredRMS = removeCoherentNoise(fullEvent,nChannelsPerGroup,fullEvent.shape[-1])
filteredFullEvent,filteredMedian,filteredRMS = removeCoherentNoiseMorphCollection(fullEvent,nChannelsPerGroup,fullEvent.shape[-1],(3,20))

waveforms = fullEvent

#structuringElement =(5,30)
structuringElement =(3,13)

median   = ndimage.median_filter(waveforms,size=4)
erosion  = ndimage.grey_erosion(waveforms,size=structuringElement)
dilation = ndimage.grey_dilation(waveforms,size=structuringElement)
opening  = ndimage.grey_opening(waveforms,size=structuringElement)
closing  = ndimage.grey_closing(waveforms,size=structuringElement)

#Reset the opening and closing baselines
#erosion  = erosion  - np.median(erosion,axis=-1)[:,None]
#dilation = dilation - np.median(dilation,axis=-1)[:,None]
#opening  = opening  - np.median(opening,axis=-1)[:,None]
#closing  = closing  - np.median(closing,axis=-1)[:,None]

gradient = dilation - erosion #closing - opening #dilation - erosion
average  = (dilation + erosion) / 2.



<div style="text-align:center;border-style: solid;border-width: 1px;">
    <h2><font color="blue"><font size="5">2D Plotting in this block</font></font></h2><br>
</div>

In [ ]:
xRange = np.arange(len(waveforms[-1]))

#Choose an event's output
wireNum = 68

print("waveforms shape:",waveforms[wireNum].shape)

figFilters = subplots.make_subplots(rows=3,cols=1,subplot_titles=("Waveform Erosion/Dilation","Waveform Opening/Closing","Waveform Gradient"),vertical_spacing=0.12)

figFilters.add_trace(go.Scatter(x=xRange,y=waveforms[wireNum],name="Waveform",opacity=0.5,line=dict(color=DEFAULT_PLOTLY_COLORS[0],width=2)),row=1,col=1)
figFilters.add_trace(go.Scatter(x=xRange,y=median[wireNum],name="median",opacity=0.5,line=dict(color=DEFAULT_PLOTLY_COLORS[1],width=1)),row=1,col=1)
figFilters.add_trace(go.Scatter(x=xRange,y=erosion[wireNum],name="erosion",line=dict(color=DEFAULT_PLOTLY_COLORS[2],width=1)),row=1,col=1)
figFilters.add_trace(go.Scatter(x=xRange,y=dilation[wireNum],name="dilation",line=dict(color=DEFAULT_PLOTLY_COLORS[3],width=1)),row=1,col=1)

figFilters.add_trace(go.Scatter(x=xRange,y=waveforms[wireNum],name="Waveform",opacity=0.5,line=dict(color=DEFAULT_PLOTLY_COLORS[0],width=2)),row=2,col=1)
figFilters.add_trace(go.Scatter(x=xRange,y=opening[wireNum],name="opening",line=dict(color=DEFAULT_PLOTLY_COLORS[1],width=2)),row=2,col=1)
figFilters.add_trace(go.Scatter(x=xRange,y=closing[wireNum],name="closing",line=dict(color=DEFAULT_PLOTLY_COLORS[2],width=2)),row=2,col=1)

figFilters.add_trace(go.Scatter(x=xRange,y=waveforms[wireNum],name="Waveform",opacity=0.5,line=dict(color=DEFAULT_PLOTLY_COLORS[0],width=1)),row=3,col=1)
figFilters.add_trace(go.Scatter(x=xRange,y=filteredFullEvent[wireNum],name="Filtered",opacity=0.5,line=dict(color=DEFAULT_PLOTLY_COLORS[4],width=1)),row=3,col=1)
figFilters.add_trace(go.Scatter(x=xRange,y=gradient[wireNum],name="Gradient",line=dict(color=DEFAULT_PLOTLY_COLORS[1],width=2)),row=3,col=1)
#figFilters.add_trace(go.Scatter(x=xRange,y=average[wireNum],name="Average",line=dict(color=DEFAULT_PLOTLY_COLORS[2],width=2)),row=3,col=1)
figFilters.add_trace(go.Scatter(x=xRange,y=erosion[wireNum],name="erosion",line=dict(color=DEFAULT_PLOTLY_COLORS[2],width=1)),row=3,col=1)
figFilters.add_trace(go.Scatter(x=xRange,y=dilation[wireNum],name="dilation",line=dict(color=DEFAULT_PLOTLY_COLORS[3],width=1)),row=3,col=1)


figFilters.update_layout(height=800,width=1000,title_text="Filtering Functions",title_font_size=30)

figFilters.show()



<div style="text-align:center;border-style: solid;border-width: 1px;">
    <h2><font color="blue"><font size="5">3D Plotting in this block</font></font></h2><br>
</div>

In [ ]:
xWires = np.arange(wireRange[0],wireRange[1],1)

yTicks = np.arange(tickRange[0],tickRange[1],(tickRange[1]-tickRange[0]+0.01)/len(xWires))

metric = np.copy(dilation[256:320])
corValArray = np.copy(waveforms[256:320])

metricRMS = np.sqrt(np.mean(np.square(metric),axis=-1))

print(metricRMS)
print(metric > metricRMS[:,None])

#filteredPeaks = np.where(metric>7.5,metric,0.) #np.zeros(metric.shape))

indices = np.where(metric>2.5*metricRMS[:,None])

#print("len first:",len(indices[0]),", len next:",len(indices[1]))
#print("first pair:",indices[0][0],",",indices[1][0])
#print("type:",type(indices),", ",type(indices[0]),", ",type(indices[1]))

wireIndices = np.where(indices[0] == 0)

indArray = np.array(indices)
print(indArray.shape,",",metric.shape)
print(indArray[0,0],",",indArray[0,1])

print(metric.shape)

corValArray[indArray[0],indArray[1]] = 0. #opening[indArray[0],indArray[1]]

print(corValArray.shape)

#print(metric[indArray[0],indArray[1]])

coherentCor = np.median(corValArray,axis=0)

waveCohCor = waveforms[256:320] - coherentCor
    
#gradPic = plotEventView3D(waveCohCor)
gradPic = plotEventView3D(filteredFullEvent)

#gradPic.add_trace(go.Scatter(x=yTicks,y=xWires,opacity=0.25,line=dict(color=DEFAULT_PLOTLY_COLORS[0],width=3)))

gradPic.show()

In [ ]:
# Let's plot the waveforms from the block above. We can use a slider so we can view
# one at a time. 

#numWaveforms = waveCohCor.shape[0]
numWaveforms = filteredFullEvent.shape[0]
baseOffset   = 0

# Make a couple of lists to contain the waveforms to display
inputWaveforms   = []
outputWaveforms  = []
closingWaveforms = []
openingWaveforms = []

# Everybody has a common x range
tickRange = np.arange(waveCohCor.shape[1])

# Add traces, one for each slider step
for step in range(numWaveforms):
    outputWaveforms.append(
        go.Scatter(
            visible=False,
            line=dict(color=DEFAULT_PLOTLY_COLORS[0], width=2),
            name="output_" + str(step),
            x=tickRange,
            y=filteredFullEvent[step,:])) #waveCohCor[step,:]))
    inputWaveforms.append(
        go.Scatter(
            visible=False,
            line=dict(color=DEFAULT_PLOTLY_COLORS[1], width=1),
            opacity=0.5,
            name="input_" + str(step),
            x=tickRange,
            y=waveforms[step+baseOffset,:]))
#    closingWaveforms.append(
#        go.Scatter(
#            visible=False,
#            line=dict(color=DEFAULT_PLOTLY_COLORS[2], width=1),
#            opacity=0.5,
#            name="dilation_" + str(step),
#            x=tickRange,
#            y=dilation[step+baseOffset,:]))
#    openingWaveforms.append(
#        go.Scatter(
#            visible=False,
#            line=dict(color=DEFAULT_PLOTLY_COLORS[3], width=1),
#            opacity=0.5,
#            name="erosion_" + str(step),
#            x=tickRange,
#            y=erosion[step+baseOffset,:]))

# Create figure
fig = go.Figure(data=inputWaveforms+outputWaveforms) #+closingWaveforms+openingWaveforms)

# Make 10th trace visible
fig.data[10].visible                = True
fig.data[10+1*numWaveforms].visible = True
#fig.data[10+2*numWaveforms].visible = True
#fig.data[10+3*numWaveforms].visible = True

# Create and add slider
steps = []
for i in range(numWaveforms):
    step = dict(
        method="restyle",
        args=["visible", [False] * len(fig.data)],
    )
    step["args"][1][i]                = True  # Toggle i'th trace to "visible"
    step["args"][1][i+1*numWaveforms] = True  # Toggle i'th trace to "visible"
#    step["args"][1][i+2*numWaveforms] = True  # Toggle i'th trace to "visible"
#    step["args"][1][i+3*numWaveforms] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "Waveform: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)

fig.show()


<div style="text-align:center;border-style: solid;border-width: 1px;">
    <h2><font color="blue"><font size="5">Run the standard coherent noise subtraction</font></font></h2><br>
</div>



In [ ]:
filteredPic = plotEventView3D(filteredFullEvent)

<div style="text-align:center;border-style: solid;border-width: 1px;">
    <h2><font color="blue"><font size="5">A Validation Block</font></font></h2><br>
</div>

<p>This can be run as a way to first check the field, electronics and then full responses created by the underlying code, and then to do a 1D overlay of a particle, once for each plane, onto a white noise generated waveform</p>

In [ ]:
import scipy.signal as signal
from sigproc_tools.sigproc_objects.fullresponse import FullResponse
from sigproc_tools.sigproc_objects.fieldresponse import FieldResponse
from sigproc_tools.sigproc_objects.electronicsresponse import ElectronicsResponse
from sigproc_tools.sigproc_functions.fakeParticle import genWhiteNoiseWaveform,genSpikeWaveform,createParticleTrajectory

# Recover x axis
respTicks = TPCresponses[2].FieldResponse.timeBins
TPCticks  = np.arange(TPCresponses[2].TPCNumTicks)

#Plot away!
figResponse = subplots.make_subplots(rows=3,cols=1,subplot_titles=("Field Response","Electronics Response","Full Response"),vertical_spacing=0.12)

figResponse.add_trace(go.Scatter(x=respTicks,y=TPCresponses[0].FieldResponse.responseVals,name="Col Field",line=dict(color=DEFAULT_PLOTLY_COLORS[0],width=1)),row=1,col=1)
figResponse.add_trace(go.Scatter(x=respTicks,y=TPCresponses[1].FieldResponse.responseVals,name="Mid Field",line=dict(color=DEFAULT_PLOTLY_COLORS[1],width=1)),row=1,col=1)
figResponse.add_trace(go.Scatter(x=respTicks,y=TPCresponses[2].FieldResponse.responseVals,name="1st Field",line=dict(color=DEFAULT_PLOTLY_COLORS[2],width=1)),row=1,col=1)

figResponse.add_trace(go.Scatter(x=respTicks,y=TPCresponses[0].ElectronicsResponse.electronicsResponse,name="Electronics",line=dict(color=DEFAULT_PLOTLY_COLORS[3],width=1)),row=2,col=1)

figResponse.add_trace(go.Scatter(x=TPCticks,y=TPCresponses[2].Response,name="Col Full",line=dict(color=DEFAULT_PLOTLY_COLORS[4],width=1)),row=3,col=1)
figResponse.add_trace(go.Scatter(x=TPCticks,y=TPCresponses[1].Response,name="Mid Full",line=dict(color=DEFAULT_PLOTLY_COLORS[5],width=1)),row=3,col=1)
figResponse.add_trace(go.Scatter(x=TPCticks,y=TPCresponses[0].Response,name="1st Full",line=dict(color=DEFAULT_PLOTLY_COLORS[6],width=1)),row=3,col=1)

figResponse.update_layout(height=800,width=1000,title_text="TPC Response Functions",title_font_size=30)

figResponse.show()


# Experiment with creating a waveform

# Start with "just" a charge deposit on a zero waveform
numElectrons = 15000

spikeResponse = [None,None,None]

spikeResponse[0],spikeWave = genSpikeWaveform(TPCresponses[0],numElectrons,1000,(4096))
spikeResponse[1],spikeWave = genSpikeWaveform(TPCresponses[1],numElectrons,1000,(4096))
spikeResponse[2],spikeWave = genSpikeWaveform(TPCresponses[2],numElectrons,1000,(4096))

# Now produce a "white noise" waveform for background
whiteRMSRaw = 7.0
whiteResponse,whiteNoise = genWhiteNoiseWaveform(TPCresponses[0],whiteRMSRaw,(3,4096))

pulseIntegral = np.sum(spikeResponse,axis=1) * 67.5

print("waveform integral:",pulseIntegral)

# Finally, add spike to noise and make a nice picture
whiteNoiseElec = spikeResponse + whiteResponse
dataNoiseElec = spikeResponse + filter.waveLessCoherentAll[0,8:11,:]


figWaveform = subplots.make_subplots(rows=3,cols=1,subplot_titles=("Input White Waveform","Output White Waveform","Data Waveform"),vertical_spacing=0.12)

figWaveform.add_trace(go.Scatter(x=TPCticks,y=spikeWave,name="Input",line=dict(color=DEFAULT_PLOTLY_COLORS[0],width=1)),row=1,col=1)
figWaveform.add_trace(go.Scatter(x=TPCticks,y=whiteNoise[0],name="Input",line=dict(color=DEFAULT_PLOTLY_COLORS[1],width=1)),row=1,col=1)
figWaveform.add_trace(go.Scatter(x=TPCticks,y=whiteNoise[1],name="Input",line=dict(color=DEFAULT_PLOTLY_COLORS[2],width=1)),row=1,col=1)
figWaveform.add_trace(go.Scatter(x=TPCticks,y=whiteNoise[2],name="Input",line=dict(color=DEFAULT_PLOTLY_COLORS[3],width=1)),row=1,col=1)

figWaveform.add_trace(go.Scatter(x=TPCticks,y=whiteNoiseElec[0],name="1st output",line=dict(color=DEFAULT_PLOTLY_COLORS[4],width=1)),row=2,col=1)
figWaveform.add_trace(go.Scatter(x=TPCticks,y=whiteNoiseElec[1],name="Mid output",line=dict(color=DEFAULT_PLOTLY_COLORS[5],width=1)),row=2,col=1)
figWaveform.add_trace(go.Scatter(x=TPCticks,y=whiteNoiseElec[2],name="Col output",line=dict(color=DEFAULT_PLOTLY_COLORS[6],width=1)),row=2,col=1)

figWaveform.add_trace(go.Scatter(x=TPCticks,y=dataNoiseElec[0],name="1st output",line=dict(color=DEFAULT_PLOTLY_COLORS[4],width=1)),row=3,col=1)
figWaveform.add_trace(go.Scatter(x=TPCticks,y=dataNoiseElec[1],name="Mid output",line=dict(color=DEFAULT_PLOTLY_COLORS[5],width=1)),row=3,col=1)
figWaveform.add_trace(go.Scatter(x=TPCticks,y=dataNoiseElec[2],name="Col output",line=dict(color=DEFAULT_PLOTLY_COLORS[6],width=1)),row=3,col=1)

figWaveform.update_layout(height=800,width=1000,title_text="Simulated Overlays",title_font_size=30)

figWaveform.show()





<div style="text-align:center;border-style: solid;border-width: 1px;">
    <h2><font color="blue"><font size="5">Compare Power Spectra of Input Data to Pure White Noise</font></font></h2><br>
</div>

In [ ]:
from sigproc_tools.sigproc_functions.fakeParticle import *
from sigproc_tools.sigproc_functions.noiseProcessing import *
from sigproc_tools.sigproc_functions.noiseAnalysis import *
from sigproc_tools.sigproc_functions.responseFunctions import *

# Define some constants
whiteRMSRaw  = 8.0

numTicks     = filter.rawdigits.numTicks(0)
tickWidth    = 0.4            # 0.4 microseconds/tick
maxFrequency = 1. / tickWidth # max frequency in MHz

waveformShape = (numChannels,numTicks)

# Create an event full of channels with white noise
whiteNoiseWaveform,whiteNoise = genWhiteNoiseWaveform(TPCresponses[2],whiteRMSRaw,waveformShape)
whiteWithField,_              = genWhiteNoiseWaveform(TPCresponses[0],whiteRMSRaw,waveformShape)

elecResponseVec = TPCresponses[2].ElecResponse
whiteNoiseVec   = np.random.normal(loc=0.,scale=whiteRMSRaw,size=numTicks)

# Little cross check here...
whiteLessPed,whitePed,whiteRMS = getPedestalsAndRMS(whiteNoiseWaveform)
checkLessPed,checkPed,checkRMS = getPedestalsAndRMS(whiteNoiseVec)

print("Cross check, white waveform RMS:",np.mean(whiteRMS,axis=-1),", raw:",checkRMS)

isItWhite = np.mean(whiteNoiseWaveform,axis=0)

figWhite = subplots.make_subplots(rows=3,cols=1,subplot_titles=("Electronics Response","White Noise","White Noise Convolved"),vertical_spacing=0.12)

figWhite.add_trace(go.Scatter(x=np.arange(numTicks),y=elecResponseVec,name="Electronics Response",line=dict(color=DEFAULT_PLOTLY_COLORS[0],width=2)),row=1,col=1)

figWhite.add_trace(go.Scatter(x=np.arange(numTicks),y=whiteNoiseVec,name="Input White Noise",line=dict(color=DEFAULT_PLOTLY_COLORS[0],width=2)),row=2,col=1)

figWhite.add_trace(go.Scatter(x=np.arange(numTicks),y=whiteLessPed[10,:],name="White Noise",line=dict(color=DEFAULT_PLOTLY_COLORS[0],width=1)),row=3,col=1)
figWhite.add_trace(go.Scatter(x=np.arange(numTicks),y=isItWhite,name="Average Noise",line=dict(color=DEFAULT_PLOTLY_COLORS[1],width=2)),row=3,col=1)

figWhite.update_layout(height=900,width=1000,title_text="Electronics Response and White Noise",title_font_size=30)
figWhite.show()

print("Now start computing the power spectra, averaged over channels/groups and over events")

freqVec, totWavePowerVec = getPowerVec(filter.waveLessPedAll,maxFrequency)
freqVec, corWavePowerVec = getPowerVec(filter.waveLessCoherentAll,maxFrequency)
freqVec, wNoisePowerVec  = getPowerVec(whiteNoiseWaveform,maxFrequency)
#freqVec, wFieldPowerVec  = getPowerVec(whiteWithField,maxFrequency)
#_,       noRespPowerVec  = getPowerVec(whiteNoise,maxFrequency)

freqVec, medianPowerVec  = getPowerVec(filter.medianAll,maxFrequency)
      

# combine the power vecs
meanTotPower = np.mean(np.mean(totWavePowerVec,axis=0),axis=0)
meanCorPower = np.mean(np.mean(corWavePowerVec,axis=0),axis=0)

meanWhiteNoise = np.mean(wNoisePowerVec,axis=0)
#meanFieldNoise = np.mean(wFieldPowerVec,axis=0)
#meanPureWNoise = np.mean(noRespPowerVec,axis=0)

meanMedPower = np.mean(np.mean(medianPowerVec,axis=0),axis=0)

print("Now plot these")

figAllFFT = go.Figure()

figAllFFT.add_trace(go.Scatter(x=freqVec,y=meanTotPower,name="Total Power",line=dict(color=DEFAULT_PLOTLY_COLORS[0],width=2)))
figAllFFT.add_trace(go.Scatter(x=freqVec,y=meanCorPower,name="Corrected Power",line=dict(color=DEFAULT_PLOTLY_COLORS[1],width=2)))
figAllFFT.add_trace(go.Scatter(x=freqVec,y=meanMedPower,name="Coherent Power",line=dict(color=DEFAULT_PLOTLY_COLORS[2],width=2)))
figAllFFT.add_trace(go.Scatter(x=freqVec,y=meanWhiteNoise,name="White noise Power",opacity=0.5,line=dict(color=DEFAULT_PLOTLY_COLORS[3],width=1)))
#figAllFFT.add_trace(go.Scatter(x=freqVec,y=meanFieldNoise,name="White w/Field Power",opacity=0.5,line=dict(color=DEFAULT_PLOTLY_COLORS[4],width=1)))
#figAllFFT.add_trace(go.Scatter(x=freqVec,y=meanPureWNoise,name="White Pure Power",opacity=0.5,line=dict(color=DEFAULT_PLOTLY_COLORS[5],width=1)))

figAllFFT.show()




<p>In the above plot we have the power spectra for the test data (blue is all, green is the coherent noise correction component, orange is the remaining component) comparing to pure white noise input. As pointed out by Mike Mooney, its not quite a fair comparison since what is "electronics noise" (some combination of the white noise from the cables passing through and contributed to by the electronics) is not well modeled by simply convolving a white noise waveform with the electronics response. Generally, the orange curve is what we will need to learn to live with...</p>